# Working with databases

- Import package mysql-connector-python https://dev.mysql.com/doc/connector-python/en/
- Create database in Xampp with script <a href="movies.sql">movies.sql</a>
<br>
<img src="database.jpg">


In [1]:
! pip install mysql-connector-python

     ---------------------------------------- 9.6/9.6 MB 4.5 MB/s eta 0:00:00
     -------------------------------------- 905.1/905.1 kB 3.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Connection database

Default user & password van phpmyadmin (Xampp) are 'root' en ''

In [5]:
import mysql.connector

db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="movies_db"
)

cursor = db.cursor()

# do something with cursor

# close the cursor
cursor.close()
db.close()

(1, 'Michelle Yeoh')
(2, 'Stephan James')
(3, 'Jamie Lee Curtis')
(4, 'Tom Cruise')


## CRUD operations

### CREATE --> add one row

In [6]:
import mysql.connector

# Connect to the database
db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="movies_db"
)

# Create a cursor object
cursor = db.cursor()

# Insert a new row into the movies table
insert_query = "INSERT INTO movies (title, synopsis) VALUES (%s, %s)"
data = ("The Matrix", "A computer hacker learns from mysterious rebels about the true nature of his reality and his role in the war against its controllers.")
cursor.execute(insert_query, data)

# Commit the transaction
db.commit()

# Close the cursor and database connection
cursor.close()
db.close()


### CREATE --> add more rows

In [7]:
import mysql.connector

# establish connection
db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="movies_db"
)

# prepare query
query = "INSERT INTO movies (title, synopsis) VALUES (%s, %s)"

# list of movies to insert
movies = [
  ("Inception", "A thief is hired to plant an idea in the mind of a corporate executive."),
  ("Blade Runner", "A detective hunts down rogue replicants in a dystopian future."),
  ("Star Wars", "A farm boy joins a rebellion to overthrow an evil empire."),
  ("Jurassic Park", "Scientists create a theme park with cloned dinosaurs, which quickly goes awry.")
]

# insert each movie
cursor = db.cursor()
for movie in movies:
  cursor.execute(query, movie)

# commit changes and close connection
db.commit()
db.close()


### READ

In [10]:
import mysql.connector

# establish connection
db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="movies_db"
)

cursor = db.cursor()

cursor.execute("SELECT * FROM movies")

movies = cursor.fetchall()

for movie in movies:
  print(movie) 		# print entire record
  print(movie[1])	# print only title of movie

# commit changes and close connection
db.commit()
db.close()

(1, 'Everything Everywhere All at Once', 'A multiverse-spanning adventure')
Everything Everywhere All at Once
(2, 'All Quiet on the Western Front', 'A World War I drama')
All Quiet on the Western Front
(3, 'The Whale', 'A man attempts to reunite with his estranged daughter')
The Whale
(4, 'Top Gun: Maverick', 'Maverick returns to train a new generation of fighter pilots')
Top Gun: Maverick
(5, 'The Matrix', 'A computer hacker learns from mysterious rebels about the true nature of his reality and his role in the war against its controllers.')
The Matrix
(6, 'Inception', 'A thief is hired to plant an idea in the mind of a corporate executive.')
Inception
(7, 'Blade Runner', 'A detective hunts down rogue replicants in a dystopian future.')
Blade Runner
(8, 'Star Wars', 'A farm boy joins a rebellion to overthrow an evil empire.')
Star Wars
(9, 'Jurassic Park', 'Scientists create a theme park with cloned dinosaurs, which quickly goes awry.')
Jurassic Park


In [12]:
import mysql.connector

# establish connection
db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="movies_db"
)

cursor = db.cursor()

cursor.execute("SELECT * FROM movies")

movies = cursor.fetchall()

# get column names
columns = [desc[0] for desc in cursor.description]

for movie in movies:
  # access fields by name
  print(movie[columns.index('title')])

# commit changes and close connection
db.commit()
db.close()

Everything Everywhere All at Once
All Quiet on the Western Front
The Whale
Top Gun: Maverick
The Matrix
Inception
Blade Runner
Star Wars
Jurassic Park


In [25]:
import mysql.connector

db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="movies_db"
)

cursor = db.cursor()

query = """SELECT movies.title, actors.name FROM movies
           JOIN movie_actor ON movies.id = movie_actor.movie_id
           JOIN actors ON movie_actor.actor_id = actors.id
           WHERE movies.id=1"""

cursor.execute(query)

results = cursor.fetchall()

for row in results:
  print(row[0]," --> ", row[1])

# close connection
db.close()


Everything Everywhere All at Once  -->  Michelle Yeoh
Everything Everywhere All at Once  -->  Stephan James
Everything Everywhere All at Once  -->  Jamie Lee Curtis


### UPDATE

In [14]:
import mysql.connector

# establish connection
db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="movies_db"
)

cursor = db.cursor()

# update the synopsis of a specific movie
update_query = "UPDATE movies SET synopsis = %s WHERE title = %s"
update_data = ("A computer hacker learns about the true nature of reality.", "The Matrix")
cursor.execute(update_query, update_data)

# commit changes and close connection
db.commit()
db.close()



### DELETE

In [4]:
import mysql.connector

# establish connection
db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="movies_db"
)

cursor = db.cursor()

# delete a specific movie
delete_query = "DELETE FROM movies WHERE title = %s"
delete_data = ("Dummy Movie",)
cursor.execute(delete_query, delete_data)

# commit changes
db.commit()

# check if movie was deleted
select_query = "SELECT * FROM movies WHERE title = %s"
select_data = ("Dummy Movie",)
cursor.execute(select_query, select_data)

if cursor.rowcount == 0:
    print("Movie was deleted successfully")
else:
    print("Movie was not deleted")

# close connection
db.close()


Movie was deleted successfully


## Scrape into database

In [12]:
from bs4 import BeautifulSoup
import requests
import mysql.connector

url = 'https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films'
r = requests.get(url)
soup = BeautifulSoup(r.text) 
table = soup.find('table',{ "class" : "wikitable sortable" }) # tabel zoeken
count=0

for tr in table.find_all('tr'):     
    row = tr.find_all('td')       
    if (len(row)==4):               
        title = row[0].text.strip().replace("'","") 
        year = row[1].text.strip()
        awards = row[2].text.strip()
        nominations = row[3].text.strip()

        # establish connection
        db = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            database="movies_db"
        )
        cursor = db.cursor()
        cursor.execute("SELECT * FROM movies where title like '" + title +"'")
        movies = cursor.fetchall()
        if len(movies)==0:
            cursor.execute("INSERT INTO movies(title) values('" + title +"')")
            
            movie_id = cursor.lastrowid
            awardshow_id = 1

            sql = "INSERT INTO awards (awardshow_id, year, movie_id, awards, nominations) VALUES (%s, %s, %s, %s, %s)"
            val = (awardshow_id, year, movie_id, awards, nominations)
            cursor.execute(sql, val)
            db.commit()

            print(title,"inserted!")
        db.close()


Guillermo del Toros Pinocchio inserted!
Navalny inserted!
The Elephant Whisperers inserted!
An Irish Goodbye inserted!
The Boy, the Mole, the Fox and the Horse inserted!
RRR inserted!
CODA inserted!
Dune inserted!
The Eyes of Tammy Faye inserted!
No Time to Die inserted!
The Windshield Wiper inserted!
The Long Goodbye inserted!
The Queen of Basketball inserted!
Summer of Soul inserted!
Drive My Car inserted!
Encanto inserted!
West Side Story inserted!
Belfast inserted!
The Power of the Dog inserted!
King Richard inserted!
Cruella inserted!
Nomadland inserted!
The Father inserted!
Judas and the Black Messiah inserted!
Minari inserted!
Mank inserted!
Sound of Metal inserted!
Ma Raineys Black Bottom inserted!
Promising Young Woman inserted!
Tenet inserted!
Soul inserted!
Another Round inserted!
My Octopus Teacher inserted!
Colette inserted!
If Anything Happens I Love You inserted!
Two Distant Strangers inserted!
Parasite inserted!
Ford v Ferrari inserted!
Learning to Skateboard in a Warzo